In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import itertools
import json
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import utils as ut
import LIM_class

plt.style.use("../plotting.mplstyle")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
data = xr.open_dataset("./data/ts_Amon_CESM2_piControl_r1i1p1f1.nc")["ts"]
#data = xr.open_dataset("./data/zos_Amon_CESM2_piControl_r1i1p1f1.nc")["zos"]
#data = xr.open_dataset("./data/ssta_1950_2021.nc")["ssta"]

data = data[:1000, :, :]
print("Data shape: {}".format(data.shape))
print("Data : {}".format(data))


# Use Principal Component Analysis to reduce the dimensionality of the data
pca_10 = ut.SpatioTemporalPCA(data, n_components=20)
eof_10 = pca_10.eofs()
pc_10 = pca_10.principal_components()

Data shape: (1000, 192, 288)
Data : <xarray.DataArray 'ts' (time: 1000, lat: 192, lon: 288)>
[55296000 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 -180.0 -178.8 -177.5 -176.2 ... 176.2 177.5 178.8
  * time     (time) object 0001-01-15 12:00:00 ... 0084-04-15 00:00:00
Attributes: (12/19)
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        Surface temperature (skin for open ocean)
    description:    Surface temperature (skin for open ocean)
    frequency:      mon
    id:             ts
    ...             ...
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Surface Temperature
    type:           real
    units:          K
    variable_id:    ts
x_proc : <xarray.DataArray 'ts' (time: 1000, z: 55296)>
array([[243.5303 , 243.53464, 243.53464, ..., 239.60129, 239.60106,
        239.60085],
       [232.6508 , 232

In [23]:
# Create training and test data
data = pca_10.principal_components()
index_train = int(0.8 * len(data["time"]))
data_train = data[:, :index_train]
data_test = data[:, index_train:]

# Creating an example LIM object
tau = 1
model = LIM_class.LIM(tau)
model.fit(data_train.data)
print("Data train : {} + shape: {} + type: {}".format(data_train.data[:5], data_train.data.shape, type(data_train.data)))

eigenvalues, _, _ = ut.matrix_decomposition(model.green_function)
t_decay = [abs(-(1/np.log(eigenvalue.real))) for eigenvalue in eigenvalues]

#print("Eigenvalues : min {} + max {}".format(min(eigenvalues), max(eigenvalues)))
#print("T-decay : min {} + max {}".format(min(t_decay), max(t_decay)))

Frobenius norm: 3.6242599487304688
Data train : [[-12131.013  -12508.043  -13168.99   ... -16400.004  -16707.822
  -16536.889 ]
 [-19987.258  -19433.979  -18764.057  ... -19822.018  -19794.018
  -19904.496 ]
 [-11377.144  -11319.862  -11080.628  ... -11533.66   -11334.262
  -11113.6875]
 [ -5021.9053  -4621.798   -5088.3076 ...  -5040.953   -5072.3975
   -5043.0195]
 [ -4725.376   -4819.449   -4731.8687 ...  -4853.042   -4698.279
   -4659.7495]] + shape: (20, 800) + type: <class 'numpy.ndarray'>


In [24]:
# Simulate stochastic differential equation
x_input = data_train.isel(time=0)
times = x_input['time']
x = x_input.data

lim_integration, times_ = model.noise_integration(x, timesteps=499, seed=10, num_comp=len(pc_10))

lim_integration = lim_integration.T
pc_10 = np.array(pc_10)
#print("LIMMM : {} + next {} + next {}".format(lim_integration[0][:5], lim_integration[0][50], lim_integration[0][-3:]))
#print("PCCC : {} + next {} + next {}".format(pc_10[0][:5], pc_10[0][50], pc_10[0][-3:] ))

t_delta: 0.3568441801550697


C:\Users\felix\PycharmProjects\ML_Climate_Science_Research_Project\LIM\LIM_class.py:239: RuntimeWarning: covariance is not positive-semidefinite.
  random_part = np.array(np.random.multivariate_normal([0 for n in range(num_comp)], self.noise_covariance))


In [25]:
from LSTM_model import *

# Set random seed for reproducibility
torch.manual_seed(42)

def hyperparameter_training_loop(data, hyperparams):

    # Generate all possible combinations of hyperparameters
    combinations = list(itertools.product(*hyperparams.values()))

    overall_best_model = {"hidden_size": None,
                         "learning_rate": None,
                         "num_epochs": None,
                         "loss": float('inf')}

    # Iterate over each hyperparameter combination
    for i, params in enumerate(combinations):
        print(f"Testing parameter combination {i+1}/{len(combinations)}: {params}")

        # Create a new model with the current hyperparameters
        hidden_size = params[0]
        learning_rate = params[1]
        num_epochs = params[2]
        num_layers = params[3]
        sequence_length = params[4]

        dataloader = create_dataloader(data, sequence_length=sequence_length, batch_size=32, shuffle=False)

        model = LSTMNetwork(1, hidden_size, num_layers)
        losses = train(model, dataloader, num_epochs, learning_rate)

        # Save the model and hyperparameters to a file
        result = {
            'hyperparameters': {
                'hidden_size': hidden_size,
                'learning_rate': learning_rate,
                'num_epochs': num_epochs,
                'num_layers': num_layers,
                'sequence_length': sequence_length,
                'best_loss': losses[-1],
                "losses": losses
            }
        }
        if losses[-1] < overall_best_model["loss"]:
            overall_best_model["hidden_size"] = params[0]
            overall_best_model["learning_rate"] = params[1]
            overall_best_model["num_epochs"] = params[2]
            overall_best_model["loss"] = losses[-1]
            overall_best_model["losses"] = losses

        filename = f"./model_trained_lstm/fnn_model_{i+1}.pt"
        with open(filename, 'w') as f:
            json.dump(result, f)

        print(f"Saved model and hyperparameters to {filename}\n")

    return overall_best_model


# Create the DataLoader for first principal component
data = np.array(data)[0]
print("Data : {} + shape: {} + type: {}".format(data, data.shape, type(data)))

# Hyperparameter search space
hyperparams = {
    'hidden_size': [32],
    'learning_rate': [0.001, 0.002],
    'num_epochs': [1000],
    'num_layers': [3],
    'sequence_length': [5, 10, 15]
}

#Train the model
#train(model, dataloader, num_epochs, learning_rate)
best_model = hyperparameter_training_loop(data, hyperparams)

print(f"Best model: {best_model}")
plot_loss_evolution(best_model["losses"], np.arange(0,best_model["num_epochs"], 100, dtype=int), best_model["learning_rate"], best_model["hidden_size"])

Data : [-12131.013  -12508.043  -13168.99   -14437.364  -15599.752  -16477.545
 -16701.354  -16649.082  -16055.64   -14854.778  -13607.776  -12565.216
 -12101.724  -12477.988  -13462.829  -14834.246  -15734.503  -16414.752
 -16618.287  -16677.6    -16066.892  -14915.94   -13507.861  -12458.042
 -12194.773  -12769.49   -13404.357  -14670.397  -15743.118  -16466.229
 -16688.477  -16513.58   -16084.348  -14809.902  -13479.406  -12629.81
 -12002.597  -12563.653  -13342.342  -14694.043  -15742.9375 -16445.773
 -16814.963  -16686.688  -15925.827  -14922.785  -13649.222  -12464.634
 -12323.74   -12632.03   -13460.442  -14702.308  -15735.301  -16414.889
 -16687.78   -16681.625  -16067.676  -15000.707  -13625.415  -12637.331
 -12281.062  -12681.287  -13458.915  -14381.694  -16004.489  -16499.076
 -16751.95   -16557.303  -16101.676  -14928.337  -13703.922  -12622.627
 -12447.257  -12631.951  -13505.499  -14578.897  -15712.284  -16506.275
 -16648.412  -16566.594  -16013.218  -14923.271  -13578.38

TypeError: only integer scalar arrays can be converted to a scalar index